In [ ]:
# default_exp blocker
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Website Blocker

> This is the core functionality of beproductive. A defined list of websites is added to the `hosts` file of the OS with a redirect to `127.0.0.1`.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
from pathlib import Path
from shutil import copy
import beproductive.config as config
import sys
try:
    from win10toast import ToastNotifier
    win_notify = ToastNotifier()
except:
    win_notify = False

In [ ]:
#export
APP_NAME = 'Be Productive'
REDIRECT = '127.0.0.1'
WIN_PATH = r'C:\Windows\System32\drivers\etc'
LINUX_PATH = r'/etc'
NOTIFY_DURATION = 1 # CHANGE TO 5 FOR PRODUCTION
ICON_PATH = 'icon.ico'

if sys.platform == 'win32':
    host_path = Path(WIN_PATH)
else:
    host_path = Path(LINUX_PATH)
host_fp = host_path/'hosts'
host_fp_copy = host_path/'hosts.original'
host_fp_blocked = host_path/'hosts.blocked'

In [ ]:
#export
class Blocker():
    "The core of the package. It modifies the hosts file of the OS."
    def __init__(self, redirect=REDIRECT):
        self.adminrights = False
        self.redirect = redirect
        self.blocklist = config.load_config()
        if not host_fp_copy.exists():
            self._setup()
        if self._create_blocked_list():
            self.adminrights = True

    def _setup(self):
        "Creates a copy of the `hosts` file and saves it as `hosts.original`"
        try:
            copy(host_fp, host_fp_copy)
            self.notify("Setup successful")
        except PermissionError:
            self._raise_permission_error()

    def _create_blocked_list(self):
        "Creates a copy of `hosts.original` and saves it to `hosts.blocked`. Then adds all blocked sites."
        try:
            copy(host_fp_copy, host_fp_blocked)
            with open(host_fp_blocked, "a") as blocked_file:
                for url in self.blocklist:
                    # TODO: refine, add www only if not in url, remove www if in url
                    blocked_file.write(f"{self.redirect} {url} www.{url} api.{url}\n")
                    # Special case for Twitter which has a special API URL
                    if url == 'twitter.com':
                        blocked_file.write(f"{self.redirect} tpop-api.twitter.com\n")
            return True
        except PermissionError:
            self._raise_permission_error()
            return False

    def block(self, notify=False):
        "Blocks all specified websites by replacing `hosts` file with `hosts.blocked`"
        try:
            copy(host_fp_blocked, host_fp)
        except PermissionError:
            self._raise_permission_error()
            return False
        if notify:
            self.notify("Websites blocked, enjoy your work.")
        return "Websites blocked"

    def unblock(self, notify=False):
        "Unblocks all websites by restoring the original `hosts` file"
        try:
            copy(host_fp_copy, host_fp)
        except PermissionError:
            self._raise_permission_error()
            return False
        if notify:
            self.notify("All websites unblocked.")
        return "Websites unblocked"

    def notify(self, message, title=APP_NAME, duration=NOTIFY_DURATION):
        "Sends notification to CLI and - if available - to GUI"
        print(message)
        if win_notify:
            win_notify.show_toast(title, message, duration=duration)

    def _raise_permission_error(self):
        self.notify("Permission Error. Please run the command line tool as ADMINISTRATOR.")

In [ ]:
show_doc(Blocker.block)

<h4 id="Blocker.block" class="doc_header"><code>Blocker.block</code><a href="__main__.py#L37" class="source_link" style="float:right">[source]</a></h4>

> <code>Blocker.block</code>(**`notify`**=*`False`*)

Blocks all specified websites by replacing `hosts` file with `hosts.blocked`

In [ ]:
show_doc(Blocker.unblock)

<h4 id="Blocker.unblock" class="doc_header"><code>Blocker.unblock</code><a href="__main__.py#L48" class="source_link" style="float:right">[source]</a></h4>

> <code>Blocker.unblock</code>(**`notify`**=*`False`*)

Unblocks all websites by restoring the original `hosts` file

In [ ]:
show_doc(Blocker.notify)

<h4 id="Blocker.notify" class="doc_header"><code>Blocker.notify</code><a href="__main__.py#L59" class="source_link" style="float:right">[source]</a></h4>

> <code>Blocker.notify</code>(**`message`**, **`title`**=*`'Be Productive'`*, **`duration`**=*`1`*)

Sends notification to CLI and - if available - to GUI

## Using Blocker

In [ ]:
import filecmp

In [ ]:
blocker = Blocker()

In [ ]:
blocker.blocklist

['twitter.com',
 'youtube.com',
 'facebook.com',
 'instagram.com',
 'reddit.com',
 'netflix.com',
 'amazon.com',
 'linkedin.com']

In [ ]:
assert filecmp.cmp(host_fp, host_fp_copy) == True, "hosts file should be identical to hosts.original"

In [ ]:
blocker.block()

'Websites blocked'

In [ ]:
assert filecmp.cmp(host_fp, host_fp_blocked) == True, "hosts file should be identical to hosts.blocked"

In [ ]:
blocker.unblock()

'Websites unblocked'

In [ ]:
assert filecmp.cmp(host_fp, host_fp_copy) == True, "hosts file should be identical to hosts.original"

In [ ]:
#hide
from nbdev.export import notebook2script; notebook2script()

Converted 00_beproductive.ipynb.
Converted 01_blocker.ipynb.
Converted 02_pomodoro.ipynb.
Converted 03_config.ipynb.
Converted index.ipynb.
